<a href="https://colab.research.google.com/github/randaama/DS4002_Group2_Project2/blob/main/SCRIPTS/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sooo we need to look at the trends of the top  10 Counties
Orange County  
Miami Dade County
Hennepin County
Hillsborough County
Broward County
Maricopa County
Hamilton County
Pinellas County
Natrona County
Fullton county
to give a desirability determination based on precipitation and average temperature. Use that to build a desiribity() function 1. Predict the average temperature and precipitation of each month based on the change in the past and then stick that into the desirability function and then rank the top Counties.

In [61]:
import pandas as pd
import plotly.express as px
#import pyearth
clim = pd.read_csv('/Users/selahmitchell/climate_data_all.csv')

In [62]:
import plotly.express as px
import numpy as np

In [63]:
clim.head()

,year,fips,month,temp,County_Name,State,percip
0,2004,1001,Jan,45.0,Autauga,Alabama,2.60
1,2005,1001,Jan,49.9,Autauga,Alabama,3.08
2,2006,1001,Jan,52.0,Autauga,Alabama,5.69
3,2007,1001,Jan,47.3,Autauga,Alabama,4.73
4,2008,1001,Jan,43.4,Autauga,Alabama,4.36


In [64]:
data = [
    {"Rank": 1, "County_Name": "Orange", "State": "Florida"},  # Adjusted to match the format
    {"Rank": 2, "County_Name": "Miami-Dade", "State": "Florida"},
    {"Rank": 3, "County_Name": "Hennepin", "State": "Minnesota"},
    {"Rank": 4, "County_Name": "Hillsborough", "State": "Florida"},
    {"Rank": 5, "County_Name": "Broward", "State": "Florida"},
    {"Rank": 6, "County_Name": "Maricopa", "State": "Arizona"},
    {"Rank": 7, "County_Name": "Hamilton", "State": "Ohio"},
    {"Rank": 8, "County_Name": "Pinellas", "State": "Florida"},
    {"Rank": 9, "County_Name": "Natrona", "State": "Wyoming"},
    {"Rank": 10, "County_Name": "Fulton", "State": "Georgia"},
]

counties_df = pd.DataFrame(data)

# Merge climate data with counties based on 'County_Name' and 'State'
top10_wallet= pd.merge(clim, counties_df, on=['County_Name', 'State'], how='inner')
top10_wallet['date'] = pd.to_datetime(top10_wallet['year'].astype(str) + '-' + top10_wallet['month'] + '-01')
print(top10_wallet)


      year   fips month  temp County_Name    State  percip  Rank       date
0     2004   4013   Jan  53.9    Maricopa  Arizona    0.52     6 2004-01-01
1     2005   4013   Jan  54.3    Maricopa  Arizona    2.45     6 2005-01-01
2     2006   4013   Jan  54.0    Maricopa  Arizona    0.00     6 2006-01-01
3     2007   4013   Jan  49.8    Maricopa  Arizona    0.68     6 2007-01-01
4     2008   4013   Jan  51.1    Maricopa  Arizona    2.19     6 2008-01-01
...    ...    ...   ...   ...         ...      ...     ...   ...        ...
2395  2019  56025   Dec  25.0     Natrona  Wyoming    0.51     9 2019-12-01
2396  2020  56025   Dec  25.5     Natrona  Wyoming    0.62     9 2020-12-01
2397  2021  56025   Dec  27.7     Natrona  Wyoming    0.41     9 2021-12-01
2398  2022  56025   Dec  21.0     Natrona  Wyoming    0.66     9 2022-12-01
2399  2023  56025   Dec  30.3     Natrona  Wyoming    0.14     9 2023-12-01

[2400 rows x 9 columns]


In [ ]:
data = [
    {"Rank": 1, "County_Name": "Maricopa", "State": "Arizona"},
    {"Rank": 2, "County_Name": "Clark", "State": "Nevada"},
    {"Rank": 3, "County_Name": "Bexar", "State": "Texas"},
    {"Rank": 4, "County_Name": "Ada", "State": "Idaho"},
    {"Rank": 6, "County_Name": "New Hanover", "State": "North Carolina"},
    {"Rank": 7, "County_Name": "Collin", "State": "Texas"},
    {"Rank": 8, "County_Name": "Miami-Dade", "State": "Florida"},
    {"Rank": 9, "County_Name": "Duval", "State": "Florida"},
    {"Rank": 10, "County_Name": "Wake", "State": "North Carolina"}
]
counties_df = pd.DataFrame(data)

# Merge climate data with counties based on 'County_Name' and 'State'
top10_migration= pd.merge(clim, counties_df, on=['County_Name', 'State'], how='inner')
print(top10_migration)


In [65]:
import pandas as pd

def calculate_weighted_weather_pattern(top10_wallet):
    # Calculate overall averages for temperature and precipitation per month across the top 10 counties
    overall_avg = top10_wallet.groupby(['month']).agg({'temp': 'mean', 'percip': 'mean'}).reset_index()
    
    # Calculate weights based on rank
    top10_wallet['weight'] = top10_wallet['Rank'].max() - top10_wallet['Rank'] + 1  # Invert rank for weighting

    # Calculate weighted averages for temperature and precipitation
    weighted_totals = top10_wallet.groupby(['month']).agg(
        weighted_temp=('temp', lambda x: (x * top10_wallet.loc[x.index, 'weight']).sum() / top10_wallet.loc[x.index, 'weight'].sum()),
        weighted_percip=('percip', lambda x: (x * top10_wallet.loc[x.index, 'weight']).sum() / top10_wallet.loc[x.index, 'weight'].sum())
    ).reset_index()

    # Merge the weighted totals back with the overall averages
    result = pd.merge(overall_avg, weighted_totals, on='month', how='left')

    return result

# Example usage:
# top10_wallet = pd.read_csv('your_data.csv')  # Load your data here
# weighted_results = calculate_weighted_weather_pattern(top10_wallet)
# print(weighted_results)




In [66]:
top10_wallet_ideal = calculate_weighted_weather_pattern(top10_wallet)

top10_wallet_ideal.head()

,month,temp,percip,weighted_temp,weighted_percip
0,Apr,63.9790,2.83375,66.405909,2.825482
1,Aug,79.9610,5.73500,80.956545,6.385500
2,Dec,51.2880,2.25730,54.629182,2.005555
3,Feb,50.9125,2.13320,54.083455,1.930082
4,Jan,48.3145,2.10445,51.348182,1.872391


In [82]:
top10_migration_ideal = calculate_weighted_weather_pattern(top10_migration)

In [83]:
fig = px.line(top10_migration_ideal, x='month', y=['temp', 'weighted_temp', 'percip', 'weighted_percip'],
              title='Monthly Temperature and Precipitation based on Migration Patterns',
              labels={'value': 'Value', 'month': 'Month'},
              markers=True)

# Show the plot
fig.show()

In [67]:
fig = px.line(top10_wallet_ideal, x='month', y=['temp', 'weighted_temp', 'percip', 'weighted_percip'],
              title='Monthly Temperature and Precipitation based on wallet',
              labels={'value': 'Value', 'month': 'Month'},
              markers=True)

# Show the plot
fig.show()

In [69]:
def calculate_yearly_desirability(monthly_averages, ideal_df, w_T=0.6, w_P=0.4):

    # Merge the ideal DataFrame with monthly averages
    merged_df = pd.merge(monthly_averages, ideal_df, on='month', suffixes=('', '_ideal'))

    # Calculate differences from ideal values
    merged_df['temp_diff'] = abs(merged_df['temp'] - merged_df['weighted_temp'])
    merged_df['percip_diff'] = abs(merged_df['percip'] - merged_df['weighted_percip'])

    # Normalize differences
    max_temp_diff = merged_df['temp_diff'].max()
    max_percip_diff = merged_df['percip_diff'].max()

    merged_df['normalized_temp_diff'] = merged_df['temp_diff'] / max_temp_diff
    merged_df['normalized_percip_diff'] = merged_df['percip_diff'] / max_percip_diff

    # Calculate monthly desirability score
    merged_df['desirability_month'] = (
        (1 - merged_df['normalized_temp_diff']) * w_T + 
        (1 - merged_df['normalized_percip_diff']) * w_P
    )

    # Calculate yearly desirability score by taking the mean of monthly scores per county and year
    yearly_desirability = merged_df.groupby(['County_Name', 'State', 'year']).agg({'desirability_month': 'mean'}).reset_index()
    yearly_desirability.columns = ['County_Name', 'State', 'year', 'desirability_year']
    top_yearly_desirability = yearly_desirability.sort_values(by='desirability_year', ascending=False).head(10)

    return top_yearly_desirability

In [87]:
top10_migration_2023 = top10_migration[top10_migration['year'] == 2023]


In [88]:
calculate_yearly_desirability(top10_migration_2023,top10_migration_ideal)

,County_Name,State,year,desirability_year
3,Collin,Texas,2023,0.879177
2,Clark,Nevada,2023,0.840474
5,Maricopa,Arizona,2023,0.794026
8,Wake,North Carolina,2023,0.783660
1,Bexar,Texas,2023,0.778729
7,New Hanover,North Carolina,2023,0.760065
4,Duval,Florida,2023,0.731268
0,Ada,Idaho,2023,0.568943
6,Miami-Dade,Florida,2023,0.488252


In [ ]:
calculate_yearly_desirability(clim_2023,top10_migration_ideal)

In [70]:
clim_2023 = clim[clim['year'] == 2023]

top10_wallet_2023 = top10_wallet[top10_wallet['year'] == 2023]


calculate_yearly_desirability(top10_wallet_2023,top10_wallet_ideal)

,County_Name,State,year,desirability_year
9,Pinellas,Florida,2023,0.815992
1,Fulton,Georgia,2023,0.815163
8,Orange,Florida,2023,0.813317
4,Hillsborough,Florida,2023,0.810245
5,Maricopa,Arizona,2023,0.778383
2,Hamilton,Ohio,2023,0.707950
6,Miami-Dade,Florida,2023,0.703147
0,Broward,Florida,2023,0.680024
3,Hennepin,Minnesota,2023,0.583719
7,Natrona,Wyoming,2023,0.480996


In [71]:

yearly_desirability_wallet_other  = calculate_yearly_desirability(clim_2023,top10_wallet_ideal)

yearly_desirability_wallet_other

In [89]:
calculate_yearly_desirability(clim_2023,top10_migration_ideal)

,County_Name,State,year,desirability_year
2733,Taylor,Texas,2023,0.957622
2306,Presidio,Texas,2023,0.957553
353,Callahan,Texas,2023,0.955550
2660,Sterling,Texas,2023,0.954236
2658,Stephens,Texas,2023,0.954068
832,Eastland,Texas,2023,0.953087
2530,Scurry,Texas,2023,0.952970
920,Fisher,Texas,2023,0.952523
2547,Shackelford,Texas,2023,0.952287
2755,Throckmorton,Texas,2023,0.952176


In [73]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Assuming final_results is your DataFrame already filtered for the year 2023
final_results = final_results[final_results['year'] == 2023]
# Create a new column for the legend
final_results['County_State'] = final_results['County_Name'] + ', ' + final_results['State']

# Create a line plot for temperature
fig_temp = px.line(
    final_results,
    x='month',
    y='temp',
    color='County_State',  # Use the new column for legend
    markers=True,
    title='Temperature in Top 10 Places to Retire (2023)'
)

# Update layout for temperature
fig_temp.update_layout(
    xaxis_title='Month',
    yaxis_title='Temperature (°F)',  # Adjust unit as needed
    legend_title='County, State',
)

# Show temperature plot
fig_temp.show()

# Create a line plot for precipitation
fig_percip = px.line(
    final_results,
    x='month',
    y='percip',
    color='County_State',  # Use the new column for legend
    markers=True,
    title='Precipitation in Top 10 Places to Retire (2023)'
)

# Update layout for precipitation
fig_percip.update_layout(
    xaxis_title='Month',
    yaxis_title='Precipitation (mm)',  # Adjust unit as needed
    legend_title='County, State',
)

# Show precipitation plot
fig_percip.show()

In [76]:

# Maybe contrast this wallethub (financially best ) with some other list of where people actually moved
#and look at the temperature differences between the 2 as welll as the "desired"
# Weight the scores based off of temp and precip 60/40
top10_wallet[['month','temp']].groupby('month').mean()

,temp
month,
Apr,63.9790
Aug,79.9610
Dec,51.2880
Feb,50.9125
Jan,48.3145
Jul,80.6195
Jun,77.7060
Mar,57.4745
May,71.0050


In [77]:
top10_wallet[['month','percip']].groupby('month').mean()

,percip
month,
Apr,2.83375
Aug,5.73500
Dec,2.25730
Feb,2.13320
Jan,2.10445
Jul,5.43905
Jun,5.52775
Mar,2.29705
May,3.61730


In [161]:
ideal_conditions = {
    'Jan': {'weighted_temp': 50.0, 'weighted_percip': 3.8},
    'Feb': {'weighted_temp': 52.5, 'weighted_percip': 4.1},
    'Mar': {'weighted_temp': 55.0, 'weighted_percip': 5.2},
    'Apr': {'weighted_temp': 62.5, 'weighted_percip': 5.0},
    'May': {'weighted_temp': 70.0, 'weighted_percip': 4.0},
    'Jun': {'weighted_temp': 80.0, 'weighted_percip': 2.3},
    'Jul': {'weighted_temp': 85.0, 'weighted_percip': 3.7},
    'Aug': {'weighted_temp': 82.5, 'weighted_percip': 2.5},
    'Sep': {'weighted_temp': 70.0, 'weighted_percip': 4.0},
    'Oct': {'weighted_temp': 62.5, 'weighted_percip': 4.5},
    'Nov': {'weighted_temp': 55.0, 'weighted_percip': 4.2},
    'Dec': {'weighted_temp': 55.0, 'weighted_percip': 3.8},
}

ideal_conditions_df = pd.DataFrame.from_dict(ideal_conditions, orient='index')

# Rename the index to represent months clearly
ideal_conditions_df.index.name = 'month'

# Display the DataFrame
print(ideal_conditions_df)


       weighted_temp  weighted_percip
month                                
Jan             50.0              3.8
Feb             52.5              4.1
Mar             55.0              5.2
Apr             62.5              5.0
May             70.0              4.0
Jun             80.0              2.3
Jul             85.0              3.7
Aug             82.5              2.5
Sep             70.0              4.0
Oct             62.5              4.5
Nov             55.0              4.2
Dec             55.0              3.8


In [162]:
calculate_yearly_desirability(clim_2023,ideal_conditions_df)

,County_Name,State,year,desirability_year
600,Columbia,Arkansas,2023,0.935430
1374,Jasper,Georgia,2023,0.933293
349,Calhoun,Mississippi,2023,0.932991
1123,Grenada,Mississippi,2023,0.932786
2031,Nevada,Arkansas,2023,0.932727
2301,Prairie,Arkansas,2023,0.932616
1834,McIntosh,Oklahoma,2023,0.931395
515,Clark,Arkansas,2023,0.931279
3081,Yalobusha,Mississippi,2023,0.931123
2593,Smith,Texas,2023,0.931030
